In [24]:
import os
import pandas as pd
import numpy as np

N_SPLIT = 10
ROOT_PATH = '/Users/ruby0322/Projects/112-1/IRTM/term-project/irtm-final-project'

In [31]:
def read_network(input_filepath, k) -> dict[list[str]]:
    G = {}
    with open(input_filepath) as file:
        while (line := file.readline()):
            if line:
                n1, n2 = line.split()
                if n1 in G:
                    G[n1].append(n2)
                else:
                    G[n1] = [n2]
                if n2 in G:
                    G[n2].append(n1)
                else:
                    G[n2] = [n1]
    return G

def split_reviews(input_filepath, output_folder) -> None:
    with open(input_filepath) as file:
        s = file.read()
        s = s.split('\n')
        slen = len(s)
        n_reviews = (slen // N_SPLIT) + 1
        for i in range(N_SPLIT):
            ending = slen if i == N_SPLIT - 1 else n_reviews*(i+1)
            ss = s[n_reviews*i:ending]
            print(f'[split-reviws] Parsing reviews split {i}...')
            reviews = dict()
            exec('\n'.join(ss), { 'reviews': reviews })
            print(reviews)
            print(f'[split-reviws] Saving reviews split {i} into "reviews-{i}.csv"...')
            pd.DataFrame(reviews).transpose().to_csv(f'{output_folder}/reviews-{i}.csv')            

def iterate_over_split_reviews(func):
    """
    func: a function that returns a list of boolean that indexes the reviews
    """
    aggr = pd.DataFrame()
    for i in range(N_SPLIT):
        df = pd.read_csv(f'./reviews/reviews-{i}.csv')
        df = df[func(df)]
        aggr = pd.concat([aggr, df])
    return aggr

def get_network_reviews(network: list[str]) -> pd.DataFrame:
    """
    network: list of users in the network
    """
    def f(df: pd.DataFrame):
        return df['user'].isin(network)
    return iterate_over_split_reviews(f)

In [32]:
def main():
    os.chdir(ROOT_PATH)
    # split_reviews('./raw-data/reviews.txt', 'reviews')
    G = read_network(f'./raw-data/edges.txt', 1)
    get_network_reviews(G['slash'] + ['slash'])

In [33]:
if __name__ == '__main__':
    main()

FileNotFoundError: [Errno 2] No such file or directory: 'reviews-0.csv'